In [24]:
import pandas as pd
import numpy as np
from numpy import loadtxt
from numpy import sort
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import PercentFormatter
import matplotlib.ticker as mtick
# from wordcloud import WordCloud, STOPWORDS 
from random import sample
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.utils import resample
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

In [25]:
ratings_df = pd.read_csv('./corporate_rating.csv')

rating_dict = {'AAA':'Lowest Risk', 
               'AA':'Low Risk',
               'A':'Low Risk',
               'BBB':'Medium Risk', 
               'BB':'High Risk',
               'B':'High Risk',
               'CCC':'Highest Risk', 
               'CC':'Highest Risk',
               'C':'Highest Risk',
               'D':'In Default'}
ratings_df['Rating'] = ratings_df['Rating'].str.strip()  # Remove leading and trailing whitespaces
ratings_df['Rating'] = ratings_df['Rating'].map(rating_dict)
ratings_df = ratings_df[ratings_df['Rating']!='Lowest Risk'] # filter Lowest Risk
ratings_df = ratings_df[ratings_df['Rating']!='In Default']  # filter In Default
ratings_df.reset_index(inplace = True, drop=True) # reset index

In [26]:
le = preprocessing.LabelEncoder()
le.fit(ratings_df.Sector)
ratings_df.Sector = le.transform(ratings_df.Sector) # encode sector
le.fit(ratings_df.Rating)
ratings_df.Rating = le.transform(ratings_df.Rating) # encode rating


In [27]:
ratings_df

,Rating,Name,Symbol,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,...,effectiveTaxRate,freeCashFlowOperatingCashFlowRatio,freeCashFlowPerShare,cashPerShare,companyEquityMultiplier,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover
0,2,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,11/27/2015,2,0.945894,0.426395,0.099690,44.203245,...,0.202716,0.437551,6.810673,9.809403,4.008012,0.049351,7.057088,15.565438,0.058638,3.906655
1,3,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,2/13/2014,2,1.033559,0.498234,0.203120,38.991156,...,0.074155,0.541997,8.625473,17.402270,3.156783,0.048857,6.460618,15.914250,0.067239,4.002846
2,3,Whirlpool Corporation,WHR,Fitch Ratings,3/6/2015,2,0.963703,0.451505,0.122099,50.841385,...,0.214529,0.513185,9.693487,13.103448,4.094575,0.044334,10.491970,18.888889,0.074426,3.483510
3,3,Whirlpool Corporation,WHR,Fitch Ratings,6/15/2012,2,1.019851,0.510402,0.176116,41.161738,...,1.816667,-0.147170,-1.015625,14.440104,3.630950,-0.012858,4.080741,6.901042,0.028394,4.581150
4,3,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,10/24/2016,2,0.957844,0.495432,0.141608,47.761126,...,0.166966,0.451372,7.135348,14.257556,4.012780,0.053770,8.293505,15.808147,0.058065,3.857790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,3,"NVR, Inc.",NVR,Moody's Investors Service,9/5/2012,1,11.757767,7.115059,7.057420,1.079762,...,0.343500,0.953231,50.293155,230.194971,1.759461,0.086376,15.867701,52.760726,0.083018,15.758348
2017,0,Kaiser Aluminum Corporation,KALU,Standard & Poor's Ratings Services,4/28/2016,1,2.962788,1.294743,0.428234,30.602414,...,0.363636,0.602645,5.563630,5.958956,1.614282,-0.267117,-4.729967,9.232021,0.114089,14.542373
2018,0,Cresud S.A.C.I.F. y A.,CRESY,Fitch Ratings,11/30/2012,6,0.883875,0.842553,0.233830,147.599371,...,35.017544,0.926665,1.459801,1.056480,4.034952,0.265092,9.358311,1.575328,0.283634,2.300168
2019,0,Cresud S.A.C.I.F. y A.,CRESY,Fitch Ratings,6/15/2012,6,0.911713,0.748356,0.310640,131.644566,...,0.331525,0.692804,0.744377,1.470201,3.825856,0.130692,22.440102,1.074441,0.217783,1.997608


In [30]:
# from sklearn import preprocessing
# min_max_scaler = preprocessing.MinMaxScaler()
# df_rating = ratings_df
# for c in df_rating.columns[6:31]:

#     df_rating[[c]] = min_max_scaler.fit_transform(df_rating[[c]].to_numpy())*1000
#     df_rating[[c]] = df_rating[[c]].apply(lambda x: np.log10(x+0.01))

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

for c in ratings_df.columns[6:31]:

    ratings_df[[c]] = min_max_scaler.fit_transform(ratings_df[[c]].to_numpy())*1000
    ratings_df[[c]] = ratings_df[[c]].apply(lambda x: np.log10(x+0.01))

In [32]:
ratings_df.describe()

,Rating,Sector,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,netProfitMargin,pretaxProfitMargin,grossProfitMargin,operatingProfitMargin,...,effectiveTaxRate,freeCashFlowOperatingCashFlowRatio,freeCashFlowPerShare,cashPerShare,companyEquityMultiplier,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover
count,2021.000000,2021.000000,2021.000000,2021.000000,2021.000000,2021.000000,2021.000000,2021.000000,2021.000000,2021.000000,...,2021.000000,2021.000000,2021.000000,2021.000000,2021.000000,2021.000000,2021.000000,2021.000000,2021.000000,2021.000000
mean,1.513112,4.977239,2.634693,2.683696,2.716262,2.757575,2.954803,2.947752,2.992425,2.938891,...,2.929743,2.988069,2.586578,1.583860,2.970580,2.947754,2.942481,2.655694,2.751430,2.716552
std,1.303252,3.464598,0.115506,0.107908,0.116308,0.107198,0.110282,0.110127,0.111113,0.109933,...,0.110181,0.111020,0.103378,0.208993,0.110624,0.110127,0.110044,0.104427,0.107037,0.106195
min,0.000000,0.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000
25%,0.000000,2.000000,2.616687,2.670570,2.683363,2.756608,2.957171,2.950097,2.994228,2.941201,...,2.932335,2.990494,2.587975,1.494723,2.973011,2.950097,2.944632,2.657097,2.750757,2.714096
50%,2.000000,5.000000,2.633601,2.681704,2.714520,2.758127,2.957188,2.950117,2.994687,2.941223,...,2.932402,2.990613,2.588008,1.529984,2.973021,2.950118,2.944667,2.657111,2.751621,2.717305
75%,3.000000,8.000000,2.654303,2.693429,2.750164,2.759412,2.957209,2.950137,2.995763,2.941246,...,2.932433,2.990674,2.588049,1.591800,2.973037,2.950139,2.944708,2.657131,2.753149,2.720481
max,3.000000,11.000000,3.000004,3.000004,3.000004,3.000004,3.000004,3.000004,3.000004,3.000004,...,3.000004,3.000004,3.000004,3.000004,3.000004,3.000004,3.000004,3.000004,3.000004,3.000004


In [ ]:
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state = 1234)

In [ ]:
X_train, y_train = train_df.iloc[:,5:31], train_df.iloc[:,0]
X_test, y_test = test_df.iloc[:,5:31], test_df.iloc[:,0]
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


In [ ]:
SVC_model = svm.SVC(kernel='rbf', gamma= 2, C = 5, random_state=1234)
SVC_model.fit(X_train, y_train)
y_pred_SVM = SVC_model.predict(X_test)
Accuracy_SVM = metrics.accuracy_score(y_test, y_pred_SVM)
print("SVM Accuracy:",Accuracy_SVM)



In [ ]:
KNN_model = KNeighborsClassifier(n_neighbors = 3)
KNN_model.fit(X_train,y_train)
y_pred_KNN = KNN_model.predict(X_test)
Accuracy_KNN = metrics.accuracy_score(y_test, y_pred_KNN)
print("KNN Accuracy:",Accuracy_KNN)

